In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp PreProcessor

# PreProcessor

> An API to Preprocess train, valid and test dataset for Machine Learning models based on tabular or strucuture data

In [3]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [4]:
# export
from tabular_ml_toolkit.DataFrameLoader import *

In [5]:
# export
# hide
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [10]:
# export

class PreProcessor:
    """
    Represent PreProcessor class
    
    Attributes:
    numerical_transformer: Numerical Columns Tranformer
    categorical_transformer: Categorical Columns Transformer
    preprocessor: Preprocessor for Columns Tranformer
    """

    def __init__(self):
        self.numerical_transformer = None
        self.categorical_transformer = None
        self.columns_transfomer = None
        self.transformer_type = None
        self.OHE_categorical_transformer = None
        self.ORE_categorical_transformer = None
#         self.numerical_cols = None
#         self.low_card_cat_cols = None
#         self.high_card_cat_cols = None
#         self.final_cols = None
    
    def __str__(self):
        """Returns human readable string reprsentation"""
        attr_str = "numerical_transformer, categorical_transformer,columns_transfomer"
        return "PreProcessor object with attributes:" + attr_str
    
    def __repr__(self):
        return self.__str__()
    
    # PreProcessor core methods
    
    # Preprocessing for numerical data
    def preprocess_numerical_data(self, num_imputer, num_scaler):
        self.numerical_transformer = Pipeline(steps=[
            ('imputer', num_imputer),
            ('scaler',  num_scaler)
        ])
        
    # Preprocessing for categorical data
    def preprocess_OHE_categorical_data(self, cat_imputer):
        self.OHE_categorical_transformer = Pipeline(steps=[
        ('imputer', cat_imputer),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])   

    def preprocess_OE_categorical_data(self, cat_imputer):
        self.OE_categorical_transformer = Pipeline(steps=[
        ('imputer', cat_imputer),
        ('ordinal', OrdinalEncoder())
        ])
        
    #TO-DO
#     def reduce_dims(self):
#         pass
    
    # Bundle preprocessing pipelines based upon types of columns
    def preprocess_all_cols(self, dataframeloader,
                            num_imputer = SimpleImputer(strategy='median'),
                            num_scaler = StandardScaler(),
                            cat_imputer = SimpleImputer(strategy='constant')):
        
        # change preprocessor according to type of column found
        
        if (dataframeloader.low_card_cat_cols and dataframeloader.high_card_cat_cols) is None:
            self.preprocess_numerical_data(num_imputer, num_scaler)
            self.transformer_type = self.numerical_transformer
        
        elif (dataframeloader.numerical_cols and dataframeloader.high_card_cat_cols) is None:
            # create scikit-learn pipelines instances
            self.preprocess_OHE_categorical_data(cat_imputer)
            # convert all categorical columns to OrdinalEncoder with Scikit-learn ColumnTranfomer
            self.columns_transfomer = ColumnTransformer(
                transformers=[
                    ('low_cad_cat', self.OHE_categorical_transformer,
                     dataframeloader.low_card_cat_cols)
                ])
            self.transformer_type = self.columns_transfomer
        
        else:
            # create scikit-learn pipelines instances
            self.preprocess_numerical_data(num_imputer, num_scaler)
            self.preprocess_OHE_categorical_data(cat_imputer)
            self.preprocess_OE_categorical_data(cat_imputer)
            # convert to Scikit-learn ColumnTranfomer
            self.columns_transfomer = ColumnTransformer(
                transformers=[
                    ('num', self.numerical_transformer,
                     dataframeloader.numerical_cols),
                    ('low_cad_cat', self.OHE_categorical_transformer,
                     dataframeloader.low_card_cat_cols),
                    ('high_cad_cat', self.OE_categorical_transformer,
                     dataframeloader.high_card_cat_cols)
                ])
            self.transformer_type = self.columns_transfomer
        
        return self

In [11]:
# showdoc(PreProcessor.preprocess_data)

#### Test PreProcessor with House Data

In [12]:
dfl = DataFrameLoader().from_csv(
    train_file_path="input/home_data/train.csv",
    test_file_path="input/home_data/test.csv",
    idx_col="Id", target="SalePrice",
    random_state=42, valid_size=0.2)
dfl

DataFrameLoader object with attributes: X_full, X_test, X(features), y(target), X_train, X_valid, y_train and y_valid

In [13]:
pp = PreProcessor().preprocess_all_cols(dataframeloader=dfl)
pp

PreProcessor object with attributes:numerical_transformer, categorical_transformer,columns_transfomer

In [14]:
# pp.columns_transfomer

In [15]:
print(len(dfl.numerical_cols))
# dfl.numerical_cols

36


In [16]:
print(len(dfl.high_card_cat_cols))
# dfl.high_card_cat_cols

3


In [17]:
print(len(dfl.low_card_cat_cols))
# dfl.low_card_cat_cols

40


In [18]:
# Define model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model

RandomForestRegressor(random_state=42)

In [19]:
# Bundle preprocessor and model in a pipeline
pl = Pipeline(steps=[('preprocessor', pp.columns_transfomer),
                      ('model', model)
                     ])
# pl

In [20]:
# for visualizing pipeline
from sklearn import set_config

set_config(display="diagram")
pl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQual...
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...]),
                                                 ('high_cad_cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Neighborhood',
                                                   'Exterior1st',
                                                   'Exterior2nd'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [21]:
# Preprocessing of training data and then fit model 
pl.fit(dfl.X_train, dfl.y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQual...
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...]),
                                                 ('high_cad_cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Neighborhood',
                                                   'Exterior1st',
                                                   'Exterior2nd'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [22]:
# Preprocessing of validation data and then get predictions
preds = pl.predict(dfl.X_valid)

print('X_valid MAE:', mean_absolute_error(dfl.y_valid, preds))

X_valid MAE: 17735.8051369863


In [23]:
# X_valid MAE: 17582.46150684932

In [24]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_DataFrameLoader.ipynb.
Converted 01_PreProcessor.ipynb.
Converted 02_MLPipeline.ipynb.
Converted 03_Tutorial.ipynb.
Converted 04_Kaggle_TPS_Challenge_Nov_2021_XGB.ipynb.
Converted 05_Kaggle_TPS_Challenge_Nov_2021_SCIKIT_LEARN.ipynb.
Converted 06_Getting_Started_Kaggle_TPS_Challenge.ipynb.
Converted 07_Kaggle_TPS_Tutorial.ipynb.
Converted 08_Optuna_Tutorial.ipynb.
Converted Untitled.ipynb.
Converted automl_in_sklearn_pipeline.ipynb.
Converted index.ipynb.
